In [3]:
#Mineral needs for dairy cows following NRC 2001
name = input('What is your name? \n')
import math

What is your name? 
 Jonathon


In [4]:
import csv
with open('info.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
    writer.writerow(['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain'])

In [5]:
#Enter basic information for the calculation
def WriteNewRow():
    cowID = input('Cow ID#: ')
    bw = input('Body weight (lb, use whole numbers): ')
    mw = float(bw)**0.75
    dim = input('Days lactating? If not a lactating cow, type "0".\n')
    dp = input('Days pregnant? If not pregnant, type "0".\n')
    breed = input('What is the breed? Type "holstein", "jersey", or "other".')
    milkprod = input('Milk yield per day (kg/day): \n')
    milkfat = input('Percent milk fat: \n')
    wg = input('Average net bodyweight gain per day (lb/day): \n')
    NewRow = cowID,bw,mw,dim,dp,breed,milkprod,milkfat,wg
    import csv
    from csv import writer
    with open('info.csv', 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(NewRow)
    import csv
    with open('info.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)

In [6]:
WriteNewRow()

Cow ID#:  1
Body weight (lb, use whole numbers):  500
Days lactating? If not a lactating cow, type "0".
 0
Days pregnant? If not pregnant, type "0".
 0
What is the breed? Type "holstein", "jersey", or "other". holstein
Milk yield per day (kg/day): 
 0
Percent milk fat: 
 0
Average net bodyweight gain per day (lb/day): 
 .04


['Cow ID', 'Body weight', 'mw', 'Days in milking', 'Days of pregnancy', 'Breed', 'Milk production', 'Milk fat', 'Weight gain']
['1', '500', '105.7371263440564', '0', '0', 'holstein', '0', '0', '.04']


In [7]:
def AddCowsQ():
    AddCows = input('Add another cow? (Y/N) \n')
    if AddCows == "Y":
        WriteNewRow()
        AddCowsQ()
    else:
        print('Ready to calculate Ca and P needs!')

In [8]:
AddCowsQ()

Add another cow? (Y/N) 
 Y
Cow ID#:  2
Body weight (lb, use whole numbers):  435
Days lactating? If not a lactating cow, type "0".
 3
Days pregnant? If not pregnant, type "0".
 170
What is the breed? Type "holstein", "jersey", or "other". holstein
Milk yield per day (kg/day): 
 43
Percent milk fat: 
 3
Average net bodyweight gain per day (lb/day): 
 .1


['Cow ID', 'Body weight', 'mw', 'Days in milking', 'Days of pregnancy', 'Breed', 'Milk production', 'Milk fat', 'Weight gain']
['1', '500', '105.7371263440564', '0', '0', 'holstein', '0', '0', '.04']
['2', '435', '95.25042951271759', '3', '170', 'holstein', '43', '3', '.1']


Add another cow? (Y/N) 
 Y
Cow ID#:  3
Body weight (lb, use whole numbers):  500
Days lactating? If not a lactating cow, type "0".
 100
Days pregnant? If not pregnant, type "0".
 200
What is the breed? Type "holstein", "jersey", or "other". holstein
Milk yield per day (kg/day): 
 54
Percent milk fat: 
 6
Average net bodyweight gain per day (lb/day): 
 .4


['Cow ID', 'Body weight', 'mw', 'Days in milking', 'Days of pregnancy', 'Breed', 'Milk production', 'Milk fat', 'Weight gain']
['1', '500', '105.7371263440564', '0', '0', 'holstein', '0', '0', '.04']
['2', '435', '95.25042951271759', '3', '170', 'holstein', '43', '3', '.1']
['3', '500', '105.7371263440564', '100', '200', 'holstein', '54', '6', '.4']


Add another cow? (Y/N) 
 N


Ready to calculate Ca and P needs!


In [8]:
#Below we will append Ca and P needs for each cow in new columns using Pandas

In [9]:
import pandas as pd
import numpy as np
import math
columns = ['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain']
Cows = pd.read_csv("./info.csv", usecols=columns)

###Ca Needs

#Urine Ca
try:
    Cows['CaUrine'] = 0.08*(Cows['Body weight'] / 100)
except:
    Cows['CaUrine'] = 'error'

#Fecal Ca
try:
    if ((Cows['Days in milking']) > 0).all():
        Cows['CaFecal'] = 3.1*(Cows['Body weight'] / 100)
    else:
        Cows['CaFecal'] = 1.54*(Cows['Body weight'] / 100)
except:
    Cows['CaFecal'] = 'error'

#Fetal Ca
try:
    if ((Cows['Days of pregnancy']) > 189).all():
        Cows['CaFetal'] = 0.02456*np.exp((0.05581-(0.00007*(Cows['Days of pregnancy'])))*(Cows['Days of pregnancy']))-0.02456*np.exp((0.05581-(0.00007*((Cows['Days of pregnancy'])-1)))*((Cows['Days of pregnancy'])-1))
    else:
        Cows['CaFetal'] = 0
except:
    Cows['CaFetal'] = 'error'

#Lactation Ca
try:
    if (Cows['Breed']).any() == 'holstein':
        Cows['CaMilkProd'] = 1.22*(Cows['Milk production'])
    elif (Cows['Breed']).any() == 'jersey':
        Cows['CaMilkProd'] = 1.45*(Cows['Milk production'])
    else:
        Cows['CaMilkProd'] = 1.37*(Cows['Milk production'])
    #print('Lactation =', round(float(CaMilkProd), 2))
except:
    Cows['CaMilkProd'] = 'error'

#Growing Ca
try:
    if ((Cows['Weight gain']) > 0).all():
        Cows['CaGrowing'] = (9.83*((Cows['mw'])**0.22)*(Cows['Body weight'])**-0.22)*((Cows['Weight gain'])/0.96)
    else:
        Cows['CaGrowing'] = 0
    #print('growing =', round(float(CaGrowing), 2))
except:
    Cows['CaGrowing'] = 'error'
       
Cows['Total Ca Need'] = Cows['CaUrine'] + Cows['CaFecal'] + Cows['CaFetal'] + Cows['CaMilkProd'] + Cows['CaGrowing']

###P Needs

#Urine P
Cows['PUrine'] = 0.002*(Cows['Body weight'])
#print('urinary =', round(float(PUrine), 2))

#Fecal P
FCM = (0.4*(Cows['Milk production']))+(15*(Cows['Milk fat'] / 100)*Cows['Milk production'])
DMI = ((0.372*(FCM))+(0.0968*(Cows['mw'])))*(1-np.exp(-0.192*((Cows['Days in milking'] / 7)+3.67)))
Cows['PFecal'] = (DMI)

#Fetal P
try:
    if (Cows['Days of pregnancy'] > 189).all():
        Cows['PFetal'] = 0.02743*np.exp((0.05527-(0.000075*(Cows['Days of pregnancy'])))*(Cows['Days of pregnancy']))-0.02743*np.exp((0.05527-(0.000075*((Cows['Days of pregnancy'])-1)))*((Cows['Days of pregnancy'])-1))
    else:
        Cows['PFetal'] = 0
except:
    Cows['PFetal'] = 'error'

#Lactation P
try:
    if (Cows['Days in milking'] > 0).all():
        Cows['PMilkProd'] = 0.9*Cows['Milk production']
    else:
        Cows['PMilkProd'] = 0
except:
    Cows['PMilkProd'] = 'error'

#Growing P
try:
    if (Cows['Weight gain'] > 0).all():
        Cows['PGrowing'] = (1.2+(4.635*((Cows['mw'])**0.22)*((Cows['Body weight'])**-0.22)))*(Cows['Weight gain'] / 0.96)
    else:
        Cows['PGrowing'] = 0
except:
    Cows['PGrowing'] = 'error'

#Total P Need
Cows['Total P Need'] = Cows['PFecal'] + Cows['PUrine'] + Cows['PFetal']+ Cows['PMilkProd'] + Cows['PGrowing']

Cows

,Cow ID,Body weight,mw,Days in milking,Days of pregnancy,Breed,Milk production,Milk fat,Weight gain,CaUrine,...,CaFetal,CaMilkProd,CaGrowing,Total Ca Need,PUrine,PFecal,PFetal,PMilkProd,PGrowing,Total P Need
0,1,500,105.737126,0,0,holstein,0,0,0.04,0.400,...,0,0.00,0.291004,8.391004,1.00,5.176157,0,0,0.187213,6.363370
1,2,435,95.250430,3,170,holstein,43,3,0.10,0.348,...,0,52.46,0.733104,60.240104,0.87,12.429653,0,0,0.470670,13.770323
2,3,500,105.737126,100,200,holstein,54,6,0.40,0.400,...,0,65.88,2.910040,76.890040,1.00,35.192916,0,0,1.872130,38.065046


In [10]:
#Create an excel doc
Cows.to_csv(r'./FinalExcel.csv', index = False)

In [16]:
#Truncate the excel doc to only include total Ca and P
columns = ['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain','Total Ca Need','Total P Need']
Cows=pd.read_csv("./FinalExcel.csv", usecols=columns)
Cows.to_csv(r'./FinalExcel.csv', index = False)
Cows

,Cow ID,Body weight,mw,Days in milking,Days of pregnancy,Breed,Milk production,Milk fat,Weight gain,Total Ca Need,Total P Need
0,1,500,105.737126,0,0,holstein,0,0,0.04,8.391004,6.363370
1,2,435,95.250430,3,170,holstein,43,3,0.10,60.240104,13.770323
2,3,500,105.737126,100,200,holstein,54,6,0.40,76.890040,38.065046


In [24]:
def DryMatter():
    Total_Ca = Cows['Total Ca Need'].sum()
    Total_P = Cows['Total P Need'].sum()
    TotalDM_Ca = Total_Ca / 0.0119
    TotalDM_P = Total_P / 0.0024
    if TotalDM_Ca > TotalDM_P:
        TotalDM_All = TotalDM_Ca
    elif TotalDM_Ca < TotalDM_P:
        TotalDM_All = TotalDM_P
    elif TotalDM_Ca == TotalDM_P:
        TotalDM_All = TotalDM_Ca
DryMatter()
print ('Total corn silage (dry matter/day, tons) needed to satisfy both Ca and P needs:',TotalDM_All*0.000001120231)

Total corn silage (dry matter/day, tons) needed to satisfy both Ca and P needs: 0.0271650126511422


In [25]:
def WetMatter():
    TotalWM_g = TotalDM_All / 0.35
    TotalWM_tons = TotalWM_g * 0.00000110231
WetMatter()
print ('Total corn silage (wet matter/day, tons) needed to satisfy both Ca and P needs:',TotalWM_tons)
print ('Total cost of needed corn silage wet matter (dollars/day):',TotalWM_tons*43.08)
print ('Totale acreage for needed corn silage (acres/year):',(TotalWM_tons / 6.6)*365)

Total corn silage (wet matter/day, tons) needed to satisfy both Ca and P needs: 0.07637267950087477
Total cost of needed corn silage wet matter (dollars/day): 3.290135032897685
Totale acreage for needed corn silage (acres/year): 4.223640608760499
